In [30]:
import sys
import os

sys.path.append(os.path.abspath("/home/miguelmd/projects/big-data-bowl-2026-analytics"))

from src.io.datasets import CSVReader
from src.core.settings import get_settings

import pandas as pd
pd.set_option('display.max_colwidth', None)

settings = get_settings()

In [31]:
players = CSVReader().read(settings.PLAYERS_FILE)
scores = CSVReader().read(settings.SCORES_FILE)
plays = CSVReader().read(settings.CLEANED_PLAYS_FILE)
inferences = CSVReader().read(settings.INFERENCE_RESULTS_FILE)

2025-12-11 17:52:23.936 | DEBUG    | src.io.datasets:read:23 - Reading data from /home/miguelmd/projects/big-data-bowl-2026-analytics/data/cleaned/players.csv
2025-12-11 17:52:23.942 | DEBUG    | src.io.datasets:read:23 - Reading data from /home/miguelmd/projects/big-data-bowl-2026-analytics/data/scores.csv
2025-12-11 17:52:23.947 | DEBUG    | src.io.datasets:read:23 - Reading data from /home/miguelmd/projects/big-data-bowl-2026-analytics/data/cleaned/plays.csv
2025-12-11 17:52:23.963 | DEBUG    | src.io.datasets:read:23 - Reading data from /home/miguelmd/projects/big-data-bowl-2026-analytics/data/processed/inference/results.csv


In [32]:
top_fakes = scores.sort_values(by="deception_score", ascending=False)
top_fakes = top_fakes.merge(plays[["game_id", "play_id", "play_description", "pass_result"]], on=["game_id", "play_id"], how="left")

In [33]:
play = top_fakes[(top_fakes['game_id'] == 2023111906) & (top_fakes['play_id'] == 2507)]
play

,game_id,play_id,defender_id,receiver_id,deception_score,recovery_score,play_description,pass_result
312,2023111906,2507,43294,55965,0.249412,0.675214,(8:22) (Shotgun) A.O'Connell pass deep middle intended for T.Tucker INTERCEPTED by J.Ramsey at MIA 44. J.Ramsey to MIA 47 for 3 yards (T.Tucker).,IN


In [34]:
bad_fake = inferences[(inferences['game_id'] == 2023111906) & (inferences['play_id'] == 2507)]
bad_fake = bad_fake.reset_index(drop=True)
bad_fake

,game_id,play_id,frame_id,receiver_id,defender_id,pass_result,non_completion_probability
0,2023111906,2507,1,55965,43294,IN,0.252428
1,2023111906,2507,2,55965,43294,IN,0.223643
2,2023111906,2507,3,55965,43294,IN,0.226390
3,2023111906,2507,4,55965,43294,IN,0.209763
4,2023111906,2507,5,55965,43294,IN,0.208982
5,2023111906,2507,6,55965,43294,IN,0.193556
6,2023111906,2507,7,55965,43294,IN,0.167096
7,2023111906,2507,8,55965,43294,IN,0.193247
8,2023111906,2507,9,55965,43294,IN,0.125037
9,2023111906,2507,10,55965,43294,IN,0.101179


In [35]:
from matplotlib.animation import PillowWriter, FuncAnimation
import matplotlib.pyplot as plt
import numpy as np
from src.metrics.deception import detect_change_point

change_point = detect_change_point(bad_fake, "non_completion_probability")

# Define colors
background_color = "#1D384B"  # Dark Gray for background
grid_color = "#9693A4"  # Light Gray for grid
line_color = "#AB1951"  # Team Red (or any bold color)
last_point_color = "#B21952"  # White for the last point to make it stand ou

non_completion_prob = bad_fake['non_completion_probability'].values
time = bad_fake['frame_id'].values

fig, ax = plt.subplots(figsize=(10, 6))
fig.patch.set_facecolor(background_color)
ax.set_facecolor(background_color)

# Title and Labels
ax.set_title('Non-Completion Probability (Pre-Throw)', fontsize=18, color='white', pad=10)
# ax.set_xlabel('Frame ID', fontsize=14)
# ax.set_ylabel('Non-Completion Probability', fontsize=16)

# Axis Limits
ax.set_xlim(time.min(), time.max()+0.5)
ax.set_ylim(-0.05, 1.05)

# Remove x-axis labels (ticks)
ax.set_xticks([])

# Y-axis ticks on the right side with white color
ax.yaxis.set_ticks_position('right')  # Move y-ticks to the right
ax.set_yticks(np.linspace(0, 1, 6))  # Explicitly set y-ticks: 0.0, 0.2, 0.4, 0.6, 0.8, 1.0
ax.tick_params(axis='y', labelcolor=grid_color, labelsize=12)

# Grid with a light gray color
ax.grid(True, linestyle='--', color=grid_color, alpha=0.6)

non_completion_line, = ax.plot(
    [], [],
    lw=5,
    color=line_color, # A deep red/orange color
    label='Non-Completion Prob',
)

last_point, = ax.plot([], [], 'o', color=last_point_color, markersize=12)
change_point_line, = ax.plot([], [], color='white', lw=2, linestyle='--')  # Vertical line at change point
break_point_label = ax.text(0, -0.01, 'Break Point', fontsize=12, color='white', ha='center', va='bottom')

# Optional: Add a Legend
# ax.legend(loc='upper right', frameon=False) 

# Add extra frames for holding the last frame longer
extra_frames = 12  # Number of extra frames to hold the last frame
extended_time = np.concatenate([time, np.full(extra_frames, time[-1])])  # Extend time with repeating last value
extended_prob = np.concatenate([non_completion_prob, np.full(extra_frames, non_completion_prob[-1])])  # Repeat last value for prob

def update(frame):
    # Update the horizontal line to the current frame
    non_completion_line.set_data(extended_time[:frame], extended_prob[:frame])

    # Ensure that last_point always receives a sequence, even if it's just one point
    last_point.set_data([extended_time[frame-1]], [extended_prob[frame-1]])

    if frame >= change_point:
        change_point_line.set_data([time[change_point], time[change_point]], [0, 1])  # Vertical line at change point
        break_point_label.set_position((time[change_point], -0.05))  # Position the label slightly below the axis
        break_point_label.set_visible(True)
    else:
        change_point_line.set_data([], [])  # Hide line before reaching the change point
        break_point_label.set_visible(False)

    return non_completion_line, last_point

# fig.tight_layout()
ax.spines['top'].set_visible(False)  # Removes the top border
ax.spines['right'].set_visible(False)  # Removes the right border
ax.spines['left'].set_visible(False)  # Removes the left border
ax.spines['bottom'].set_visible(False)  # Removes the bottom border
ani = FuncAnimation(
    fig, 
    update, 
    frames=len(extended_time),  # Use extended time array to hold the last frame
    blit=True, 
    interval=100  # Default interval, can be adjusted for smoothness
)

# Save the GIF (you may want to increase fps for a smoother result)
ani.save("2023111906_2507_non_completion.gif", writer=PillowWriter(fps=10, bitrate=1800))

# Close the figure to free up memory
plt.close(fig)
print("Animation saved as '2023111906_2507_non_completion.gif'")

Animation saved as '2023111906_2507_non_completion.gif'
